In [1]:
import mediapy as media
import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp
import mujoco
import numpy as np
from mujoco import mjx

In [2]:
from mujoco_playground import registry
env = registry.load('CartpoleBalance')
env

In [3]:
env_cfg = registry.get_default_config('CartpoleBalance')
env_cfg

action_repeat: 1
ctrl_dt: 0.01
episode_length: 1000
sim_dt: 0.01
vision: false
vision_config:
  enabled_geom_groups:
  - 0
  - 1
  - 2
  gpu_id: 0
  history: 3
  render_batch_size: 512
  render_height: 64
  render_width: 64
  use_rasterizer: false

In [4]:
env.action_size, env.observation_size

(1, 5)

In [5]:
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)

In [6]:
type(env)

mujoco_playground._src.dm_control_suite.cartpole.Balance

In [7]:
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state]

f = 0.5
for i in range(env_cfg.episode_length):
  action = []
  for j in range(env.action_size):
    action.append(
        jnp.sin(
            state.data.time * 2 * jnp.pi * f + j * 2 * jnp.pi / env.action_size
        )
    )
  action = jnp.array(action)
  state = jit_step(state, action)
  rollout.append(state)

frames = env.render(rollout)
media.show_video(frames, fps=1.0 / env.dt)

100%|██████████| 1001/1001 [00:11<00:00, 89.57it/s]


This browser does not support the video tag.


In [13]:
prev_obs = 0
transition_0 = rollout[0]
action = 0
log_prob = 0.0
0, action, transition_0.reward, transition_0.obs, transition_0.done, log_prob

(0,
 0,
 Array(0., dtype=float32),
 Array([ 0.06846283,  0.99943876, -0.03349775, -0.02442456, -0.02035681],      dtype=float32),
 Array(0., dtype=float32),
 0.0)

In [10]:
help(rollout[0])

Help on State in module mujoco_playground._src.mjx_env object:

class State(builtins.object)
 |  State(data: mujoco.mjx._src.types.Data, obs: Union[jax.Array, Mapping[str, jax.Array]], reward: jax.Array, done: jax.Array, metrics: Dict[str, jax.Array], info: Dict[str, Any]) -> None
 |  
 |  Environment state for training and inference.
 |  
 |  Methods defined here:
 |  
 |  __delattr__(self, name)
 |      Implement delattr(self, name).
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __init__(self, data: mujoco.mjx._src.types.Data, obs: Union[jax.Array, Mapping[str, jax.Array]], reward: jax.Array, done: jax.Array, metrics: Dict[str, jax.Array], info: Dict[str, Any]) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value)
 |      Implement setattr(self, name, value).
 |  
 |  replace(self, **updates) from fl